In [ ]:
import os
import io
import sys
from dotenv import load_dotenv
from google import genai
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
load_dotenv(override=True)

In [ ]:
GEMINI_API_KEY = os.getenv(key="GEMINI_API_KEY")

if GEMINI_API_KEY:
    print("GEMINI_API_KEY found.")
else:
    raise "API Key not found."

gemini = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
GEMINI_2_0_MODEL = "gemini-2.0-flash"

ACTIVE_MODEL = GEMINI_2_0_MODEL

In [ ]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [ ]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [ ]:
def messages_for(python):
    return [{"role": "user", "parts": [{"text": user_prompt_for(python)}]}]

In [ ]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("code_converter/cpp/optimized_workspace.cpp", "w") as f:
        f.write(code)

In [ ]:
def optimize_code(python):
    response = gemini.models.generate_content(
      model=ACTIVE_MODEL,
      contents= messages_for(python),
      config={"system_instruction": {"text": system_message}}
    )

    write_output(response.candidates[0].content.parts[0].text)

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

- Result: 3.141592658589
- Execution Time: 8.595519 seconds

In [ ]:
optimize_code(pi)

In [ ]:
!clang++ -std=c++17 -march=armv8.3-a -o code_converter/cpp/optimized_workspace code_converter/cpp/optimized_workspace.cpp
!./code_converter/cpp/optimized_workspace

- Result: 3.141592658589
- Execution Time: 0.698493 seconds

In [ ]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_sub_array_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_sub_array_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_sub_array_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_sub_array_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Sub-array Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
exec(python_hard)

- Total Maximum Subarray Sum (20 runs): 10980
- Execution Time: 24.565041 seconds

In [ ]:
optimize_code(python_hard)

In [ ]:
!clang++ -std=c++17 -march=armv8.3-a -o code_converter/cpp/optimized_workspace code_converter/cpp/optimized_workspace.cpp
!./code_converter/cpp/optimized_workspace

- Total Maximum Subarray Sum (20 runs): 3480
- Execution Time: 2.229859 seconds